In [ ]:
! pip install -U sentence-transformers

In [ ]:
! python3 -m pip install --upgrade xlrd

In [21]:

from sentence_transformers import SentenceTransformer, util
import numpy as np
model = SentenceTransformer('stsb-roberta-large', device='cpu')

In [27]:
from transformers.models.decision_transformer.modeling_decision_transformer import DecisionTransformerGPT2PreTrainedModel
import pandas as pd

#used as an example from code Tosin pushed on Github, needs to be changed during integration
extractedLocations = ['Cambodia','Cambodia','Ngan Chanta','Cambodia','Cambodia','D ecember-1997','Cambodia','Cambodi','Phnom Penh','Kampo', 'Phnom Penh', 'Kampong Sp eu','Kampong Cham', 'Phnom Penh', 'Cambodia', 'Kandal', 'Banthey', 'Phnom Penh','Banthey', 'Meanchey', 'Kampong Cham', 'Phnom Penh', 'Kampong Cham','Phnom Penh', 'Phnom Penh', 'Phnom Penh', 'Bangkok', 'Kuala Lumpur']


def FindCosSim(word1, word2):
  # encode sentences to get their embeddings
  embedding1 = model.encode(word1, convert_to_tensor=True)
  embedding2 = model.encode(word2, convert_to_tensor=True)
  # compute similarity scores of two embeddings
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

  return (cosine_scores.item())

def getAdmin0Code(loc):
  bestCosSimScore = 0
  row_num = 0
  int_row_num = 0

  df0 = pd.read_excel('admin0_codes.xlsx')
  # if first letter of attributes.gis_name(country name) starts with the loc first letter
  df0_filtered = df0[(df0['attributes_iso3'] != 'AAA') & (df0['attributes.gis_name'].str[0] == loc[0])]

  # # DEBUG: 
  # print("df0 is: " + str(df0))
  # print("df0 filtered is: " + str(df0_filtered))
  code = None
  # try find exact match on spreadsaheet, if there, return the ISO code of it
  if loc in df0_filtered['attributes.gis_name'].values:
    row_num = df0[df0['attributes.gis_name'] == loc].index.to_numpy()
    int_row_num = int(row_num[0])
    code = df0.iat[int_row_num, 3]
    return (code)

  else:
    bestCosSimScore = 0
    for i in df0_filtered['attributes.gis_name'].values:
      score = FindCosSim(loc, i)
      if score > bestCosSimScore:
        bestCosSimScore = score
        row_num = df0[df0['attributes.gis_name'] == i].index.to_numpy()
        int_row_num = int(row_num[0])
        code = df0.iat[int_row_num, 2]
    
    print(code)

# getAdmin0Code('colombia')


# for admin 1 & 2
def mapToISOCodes(loc, admin0Code):
  bestCosSimScore = 0
  row_num = 0
  int_row_num = 0

  df1 = pd.read_excel('src/Backend/Integration/admin1_codes.xlsx')
  df2 = pd.read_excel('src/Backend/Integration/admin2_codes.xlsx')

  # return 1 shows its admin 1, return 2 shows its admin 2
  if loc in df1['attributes.gis_name'].values:
    row_num = df1[df1['attributes.gis_name'] == loc].index.to_numpy()
    int_row_num = int(row_num[0])
    code = df1.iat[int_row_num, 3]
    return ("1",code)

  elif loc in df2['attributes.gis_name'].values:
    row_num = df2[df2['attributes.gis_name'] == loc].index.to_numpy()
    int_row_num = int(row_num[0])
    code = df2.iat[int_row_num, 3]
    return ("2",code)

  df1_filtered = df1[(df1.attributes_iso3 == admin0Code) & (df1['attributes.gis_name'].str[0] == loc[0])]
  df2_filtered = df2[(df2.attributes_iso3 == admin0Code) & (df2['attributes.gis_name'].str[0] == loc[0])]

  bestCosSimScore1 = 0
  for i in df1_filtered['attributes.gis_name'].values:
    score1 = FindCosSim(loc, i)
    if score1 > bestCosSimScore1:
      bestCosSimScore1 = score1
      row_num = df1[df1['attributes.gis_name'] == i].index.to_numpy()
      int_row_num = int(row_num[0])
      code1 = df1.iat[int_row_num, 3]

  bestCosSimScore2 = 0
  for i in df2_filtered['attributes.gis_name'].values:
    score2 = FindCosSim(loc, i)
    if score2 > bestCosSimScore2:
      bestCosSimScore2 = score2
      row_num = df2[df2['attributes.gis_name'] == i].index.to_numpy()
      int_row_num = int(row_num[0])
      code2 = df2.iat[int_row_num, 3]    

  if bestCosSimScore1 > bestCosSimScore2:
    return ("1",code1)
  return ("2",code2)


print("Code:", mapToISOCodes('Zunya', 'CHN'))

FileNotFoundError: [Errno 2] No such file or directory: 'src/Backend/Integration/admin1_codes.xlsx'